In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams.update({'font.size':20})
plt.rcParams["figure.figsize"] = (12,8)

In [2]:
str_files = ''
#Carrega os dados do arquivo .csv
df = pd.read_csv(str_files + "cetesb_concatenado06semoutliers.csv",encoding='utf-8',sep=',',index_col=[0])
df

,ph,coliformes,dbo,fosforo,od,solido,temperatura,turbidez
0,0.870126,6.173772,1.320815,0.236840,0.359643,2.465836,1.349063,1.801834
1,0.869470,6.202977,1.302178,0.231677,0.373142,2.465102,1.349748,1.804123
2,0.868813,6.230341,1.282705,0.226451,0.386234,2.464367,1.350432,1.806400
3,0.868155,6.256082,1.262318,0.221162,0.398944,2.463630,1.351114,1.808665
4,0.867496,6.280383,1.240926,0.215808,0.411292,2.462892,1.351795,1.810919
...,...,...,...,...,...,...,...,...
498,0.921166,5.909988,1.621028,0.119242,0.152288,2.538675,1.357197,1.518514
499,0.922725,5.967976,1.435367,0.188061,0.235528,2.528631,1.422631,1.555475
500,0.893595,6.019126,1.104292,0.247452,0.512718,2.518349,1.399241,1.669450
501,0.898643,6.019126,1.104292,0.247452,0.512718,2.518349,1.346108,1.669450


In [3]:
'''
def seleciona_colunas(trainingd):
    fdbo     = trainingd.iloc[:,2:3].values
    #ffosforo = trainingd.iloc[:,2:3].values
    fph      = trainingd.iloc[:,0:1].values
    fsolido  = trainingd.iloc[:,5:6].values
    
    #frame_completo = pd.DataFrame(list(zip(fdbo, ffosforo,fsolido,fph)),columns =['dbo', 'fosforo','solido','ph']) 
    frame_completo = pd.DataFrame(list(zip(fdbo,fsolido,fph)),columns =['dbo','solido','ph']) 
    
    return frame_completo
'''
# OBS.: O parâmetro temperatura não possui correlação suficiente para este tipo de experimento.
def seleciona_colunas(trainingd,p):
    
    posicao_coluna = 0
    frame_completo = pd.DataFrame()
    
    if (p == 'ph'):
        fcoliformes = trainingd.iloc[:,1:2].values
        fdbo        = trainingd.iloc[:,2:3].values
        fph         = trainingd.iloc[:,0:1].values
        fsolido     = trainingd.iloc[:,5:6].values
        
        frame_completo = pd.DataFrame(list(zip(fcoliformes,fdbo,fsolido,fph)),columns =['coliformes','dbo','solido','ph']) 
        posicao_coluna = 3
        
    elif (p == 'od'):
        fod      = trainingd.iloc[:,4:5].values
        fsolido  = trainingd.iloc[:,5:6].values
    
        frame_completo = pd.DataFrame(list(zip(fsolido,fod)),columns =['solido','od']) 
        posicao_coluna = 1
        
    elif (p == 'coliformes'):
        fph          = trainingd.iloc[:,0:1].values
        fcoliformes  = trainingd.iloc[:,1:2].values
      
        frame_completo = pd.DataFrame(list(zip(fph,fcoliformes)),columns =['ph','coliformes']) 
        posicao_coluna = 1
        
    elif (p == 'dbo'):
        #ph, fosforo, solido tem correlação com o dbo
        fph      = trainingd.iloc[:,0:1].values
        fdbo     = trainingd.iloc[:,2:3].values
        ffosforo = trainingd.iloc[:,3:4].values
        fsolido  = trainingd.iloc[:,5:6].values
      
        frame_completo = pd.DataFrame(list(zip(fph,ffosforo,fsolido,fdbo)),columns =['ph','fosforo','solido','dbo']) 
        posicao_coluna = 3
        
    elif (p == 'fosforo'):
        # dbo e solido tem correlação com o fosforo
        fdbo     = trainingd.iloc[:,2:3].values
        ffosforo = trainingd.iloc[:,3:4].values
        fsolido  = trainingd.iloc[:,5:6].values
      
        frame_completo = pd.DataFrame(list(zip(fdbo,fsolido,ffosforo)),columns =['dbo','solido','fosforo']) 
        posicao_coluna = 2
        
    elif (p == 'solido'):
        # ph, dbo, fosforo, od, solido e turbidez tem correlação com o solido
        fph        = trainingd.iloc[:,0:1].values
        fdbo       = trainingd.iloc[:,2:3].values
        ffosforo   = trainingd.iloc[:,3:4].values
        fod        = trainingd.iloc[:,4:5].values
        fsolido    = trainingd.iloc[:,5:6].values
        fturbidez  = trainingd.iloc[:,7:8].values
      
        frame_completo = pd.DataFrame(list(zip(fph,fdbo,ffosforo,fod,fturbidez,fsolido)),
                                  columns =['ph','dbo','fosforo','od','turidez','solido']) 
        posicao_coluna = 5
        
    elif (p == 'turbidez'):
        # solido e turbidez tem correlação com o turbidez
    
        fsolido    = trainingd.iloc[:,5:6].values
        fturbidez  = trainingd.iloc[:,7:8].values
      
        frame_completo = pd.DataFrame(list(zip(fsolido,fturbidez)),columns =['solido','turidez']) 
        posicao_coluna = 1
        
    return frame_completo,posicao_coluna

'''
def seleciona_colunas_od(trainingd):
    fdbo     = trainingd.iloc[:,2:3].values
    #ffosforo = trainingd.iloc[:,2:3].values
    fod      = trainingd.iloc[:,4:5].values
    fsolido  = trainingd.iloc[:,5:6].values
    
    #frame_completo = pd.DataFrame(list(zip(fdbo, ffosforo,fsolido,fph)),columns =['dbo', 'fosforo','solido','ph']) 
    frame_completo = pd.DataFrame(list(zip(fdbo,fsolido,fod)),columns =['dbo','solido','od']) 
    
    return frame_completo

def seleciona_colunas_coliformes(trainingd):
    #ph tem correlação com o coliformes
    fph          = trainingd.iloc[:,0:1].values
    fcoliformes  = trainingd.iloc[:,1:2].values
      
    frame_completo = pd.DataFrame(list(zip(fph,fcoliformes)),columns =['ph','coliformes']) 
    
    return frame_completo

def seleciona_colunas_dbo(trainingd):
    #ph, fosforo, solido tem correlação com o dbo
    fph      = trainingd.iloc[:,0:1].values
    fdbo     = trainingd.iloc[:,2:3].values
    ffosforo = trainingd.iloc[:,3:4].values
    fsolido  = trainingd.iloc[:,5:6].values
      
    frame_completo = pd.DataFrame(list(zip(fph,fdbo,ffosforo,fsolido)),columns =['ph','dbo','fosforo','solido']) 
    
    return frame_completo

def seleciona_colunas_fosforo(trainingd):
    # dbo e solido tem correlação com o fosforo
    fdbo     = trainingd.iloc[:,2:3].values
    ffosforo = trainingd.iloc[:,3:4].values
    fsolido  = trainingd.iloc[:,5:6].values
      
    frame_completo = pd.DataFrame(list(zip(fdbo,ffosforo,fsolido)),columns =['dbo','fosforo','solido']) 
    
    return frame_completo

def seleciona_colunas_solido(trainingd):
    # ph, dbo, fosforo, od, solido e turbidez tem correlação com o solido
    fph        = trainingd.iloc[:,0:1].values
    fdbo       = trainingd.iloc[:,2:3].values
    ffosforo   = trainingd.iloc[:,3:4].values
    fod        = trainingd.iloc[:,4:5].values
    fsolido    = trainingd.iloc[:,5:6].values
    fturbidez  = trainingd.iloc[:,7:8].values
      
    frame_completo = pd.DataFrame(list(zip(fph,fdbo,ffosforo,fod,fsolido,fturbidez)),
                                  columns =['ph','dbo','fosforo','od','solido','turidez']) 
    
    return frame_completo

def seleciona_colunas_turbidez(trainingd):
    # solido e turbidez tem correlação com o turbidez
    
    fsolido    = trainingd.iloc[:,5:6].values
    fturbidez  = trainingd.iloc[:,7:8].values
      
    frame_completo = pd.DataFrame(list(zip(fsolido,fturbidez)),
                                  columns =['solido','turidez']) 
    
    return frame_completo
    
'''

def pegar_dados_coluna_predita_train_test(trainingd,percent,index_coluna):
    data = trainingd.iloc[:,:].values
    train = trainingd.iloc[0:int(len(data)*percent),:].values  
    train_previsao = trainingd.iloc[0:int(len(data)*percent),index_coluna:index_coluna+1].values
    test = trainingd.iloc[len(train):,index_coluna:index_coluna+1].values
    
    return train,train_previsao, test

#Normalização dos dados: Normaliza os dados dentro um intervalo (0 a 1).
def normalizacao(train,test):
    sc = MinMaxScaler()
    testd = test
    train = sc.fit_transform(train)
    test = sc.fit_transform(test)    
    return train,test,testd

#Prepara o conjunto de dados em X e y, considerando a janela de visualização (lags).
#cy = coluna que será predita
def prepara_dados(dados,lags,cy):
    X = []
    y = []
    for i in range(lags, len(dados)):
        X.append(dados[i-lags:i,:])
        y.append(dados[i, cy])
       
    return np.array(X), np.array(y)

#Calcula o MAPE
#Define função para calcular o MAPE
#def mape(y_pred,y_true):
 #   return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mape(y_pred,y_true):
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    '''  
    mape_sum = 0
    for real,prediction in zip(y_true,y_pred):
        mape_sum += (abs((real - prediction))/real)
        
        print(real)
        mape = mape_sum/len(real)
    '''
    return mape

def rmse(y_pred,y_true):
    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    return rmse

def r2(y_pred,y_true):
    rscore = r2_score(y_true,y_pred)
    return rscore

def rquadrado(y_pred,y_true):
    #Soma Total dos Quadrados (STQ): mostra a variação de y em torno da própria média. 
    #É o somatório das diferenças entre o valor alvo real e sua média elevado ao quadrado.
    y_traco = np.mean(y_true)
    
    print('media y_true:', y_traco)
        
    stq = 0
    
    for s in y_true:
        a = s - y_traco
        st = a * a
        stq = stq + st
    
    print('Soma Total dos Quadrados (STQ):', stq[0])
    
    #Soma dos Quadrados dos Resíduos (SQU): variação de Y que não é explicada pelo modelo elaborado. 
    #É o somatório das diferenças entre o valor predito e o valor real elevados ao quadrado.
    squ = 0
    
    y_traco = np.mean(y_pred)
    print('media y_pred:', y_traco)
    
    for n in range(len(y_true)):
        a = y_true[n] - y_pred[n]
        st = a * a
        squ = squ + st
        
    print('Soma dos Quadrados dos Resíduos (SQU):',squ[0])
    print('\n')
    print('Fórmula do R²')
    print('\n')
    print('sqr = stq - squ')
    sqr = stq[0] - squ[0]
    print('R² = sqr/stq')
    sqr = sqr/stq[0]
    print('\n')
    return sqr

def correlacao_determinacao(dtframe,tipo):
    
    if (tipo == 0): #Treino
        resultado = dtframe.corr().previsao_treino.values[1] ** 2 
    else: #teste
        resultado = dtframe.corr().previsao_teste.values[1] ** 2
    
    return resultado

def ajusta_array(array):
    lista = []
    
    for i in range(len(array)):
        lista.append(array[i][0])
        
    #print('ajusta array:',lista)
        
    return lista

def ajusta_lista(array):
    lista = []
    
    for i in range(len(array)):
        lista.append(array[i][0])
        
    #print('ajusta array:',lista)
        
    return lista


In [4]:
df_resultado_medio_g = pd.DataFrame()

index_coluna = 0

parametro = []

#média
media_parametro_mape_treino = []
media_parametro_rmse_treino = []
media_parametro_r_treino    = []
    
media_parametro_mape_teste  = []
media_parametro_rmse_teste  = []
media_parametro_r_teste     = []
    
#desvio_padrão
    
std_parametro_mape_treino = []
std_parametro_rmse_treino = []
std_parametro_r_treino    = []
    
std_parametro_mape_teste  = []
std_parametro_rmse_teste  = []
std_parametro_r_teste     = []

for p in ('ph','coliformes', 'dbo','fosforo','od','solido','turbidez'):
    
    print('Agora é a vez do parâmetro:', p)
   
    trainingd,posicao_col = seleciona_colunas(df,p)
    
    #print(posicao_col)
   
    index_coluna = index_coluna + 1

    df_resultados_treino = pd.DataFrame()
    df_resultados_teste  = pd.DataFrame()
    df_resultados        = pd.DataFrame()
    
    media_lag_mape_treino = []
    media_lag_rmse_treino = []
    media_lag_r_treino    = []
        
    media_lag_mape_teste = []
    media_lag_rmse_teste = []
    media_lag_r_teste    = []

    for l in range(1, 16):
    
        lags = l
    
        #seleciona os dados
    
        train,train_previsao, test = pegar_dados_coluna_predita_train_test(trainingd,0.70,posicao_col) 
    
        #normalização dos dados
        train,test,testd = normalizacao(train,test)
        normalizador_previsao = MinMaxScaler()
        sc = MinMaxScaler()
        normalizador_previsao.fit_transform(train_previsao)
    
        #Prepara os dados de treinamento
        #Vai predizer o valor da coluna 4 (valor ph)
        train_X, train_y = prepara_dados(train, lags,posicao_col)
        
        #Prepara os dados de teste
        entradas = trainingd[len(trainingd) - len(test) - lags:].values
        entradas = sc.fit_transform(entradas)   
    
        test_X = []
        for i in range(lags, lags+len(test)):
            test_X.append(entradas[i-lags:i, 0:posicao_col+1])
        test_X = np.array(test_X)
    
        #Ajusta a dimensão de test_X e train_X
        nsamples, nx, ny = test_X.shape
        test_X = test_X.reshape((nsamples,nx*ny))
    
        nsamples, nx, ny = train_X.shape
        train_X = train_X.reshape((nsamples,nx*ny))
    
        #Cria o modelo
        model = RandomForestRegressor()
    
        #Treina o modelo
        model.fit(train_X,train_y)
        score = model.score(train_X,train_y)
        #print(score)

        #Dados de teste
        previsoes = model.predict(test_X)
        previsoes = previsoes.reshape(-1, 1)
        previsoes = normalizador_previsao.inverse_transform(previsoes)
    
        '''
        print('Teste - Gráficos com lag', l)
        #Plotagem do gráfico
        plt.plot(testd,color='red',label = 'Observado')
        plt.plot(previsoes,color='blue',label = 'Previsoes')
        plt.xlabel('Tempo')
        plt.ylabel('Valor pH')
        plt.legend()
        plt.show()
        '''
    
        #Dados de treino
        previsoes_treino = model.predict(train_X)
        previsoes_treino = previsoes_treino.reshape(-1, 1)
        previsoes_treino = normalizador_previsao.inverse_transform(previsoes_treino)
    
        '''
        print('Treinamento - Gráficos com lag', l)
        #Plotagem do gráfico
        plt.plot(train_previsao,color='red',label = 'Observado')
        plt.plot(previsoes_treino,color='blue',label = 'Previsoes')
        plt.xlabel('Tempo')
        plt.ylabel('Valor pH')
        plt.legend()
        plt.show()
        '''
        
        treino = train_previsao[lags: len(previsoes_treino) + lags, :]
        observado_test = testd
    
        observado_treino = train_previsao
    
        treino         = ajusta_array(treino)
        observado_test = ajusta_array(observado_test)
               
        #Calculo do erro da previsão MAPE, RMSE e R²
    
        observado_treino_d = observado_treino
        previsoes_treino_d = previsoes_treino
        previsoes_d        = previsoes
        
        mape_treino_d        = round(mape(previsoes_treino,treino),4)
        rmse_treino_d        = round(rmse(previsoes_treino,treino),4)
        #r_treino_d           = round(r2(previsoes_treino,treino),4)
        
        mape_teste_d        = round(mape(previsoes,observado_test),4)
        rmse_teste_d        = round(rmse(previsoes,observado_test),4)
        #r_teste_d           = round(r2(previsoes,observado_test),4) 
       
        df_corr_determinacao_treino = pd.DataFrame()
        df_corr_determinacao_teste  = pd.DataFrame()
    
        #Calcula o coeficiente de determinação
        dict = {'previsao_treino': ajusta_lista(previsoes_treino), 'treino': ajusta_array(treino)} 
    
        df_treino = pd.DataFrame(dict)
        dframes_treino = [df_corr_determinacao_treino,df_treino]
        df_corr_determinacao_treino = pd.concat(dframes_treino)
    
        dict = {'previsao_teste': ajusta_lista(previsoes), 'teste': ajusta_array(observado_test)} 
    
        df_teste = pd.DataFrame(dict)
        dframes_teste = [df_corr_determinacao_teste,df_teste]
        df_corr_determinacao_teste = pd.concat(dframes_teste)
    
        r_treino_d = []
        r_teste_d  = []
    
        r_treino_d.append(round(correlacao_determinacao(df_corr_determinacao_treino,0),2))
        r_teste_d.append(round(correlacao_determinacao(df_corr_determinacao_teste,1),2))
        
        media_lag_mape_treino.append(np.mean(mape_treino_d))
        media_lag_rmse_treino.append(np.mean(rmse_treino_d))
        media_lag_r_treino.append(np.mean(r_treino_d))
        
        media_lag_mape_teste.append(np.mean(mape_teste_d))
        media_lag_rmse_teste.append(np.mean(rmse_teste_d))
        media_lag_r_teste.append(np.mean(r_teste_d))
        
                
    #média
    media_parametro_mape_treino.append(np.mean(media_lag_mape_treino))
    media_parametro_rmse_treino.append(np.mean(media_lag_rmse_treino))
    media_parametro_r_treino.append(np.mean(media_lag_r_treino))
    
    media_parametro_mape_teste.append(np.mean(media_lag_mape_teste))
    media_parametro_rmse_teste.append(np.mean(media_lag_rmse_teste))
    media_parametro_r_teste.append(np.mean(media_lag_r_teste))
        
    #desvio_padrão
    
    std_parametro_mape_treino.append(np.std(media_lag_mape_treino))
    std_parametro_rmse_treino.append(np.std(media_lag_rmse_treino))
    std_parametro_r_treino.append(np.std(media_lag_r_treino))
    
    std_parametro_mape_teste.append(np.std(media_lag_mape_teste))
    std_parametro_rmse_teste.append(np.std(media_lag_rmse_teste))
    std_parametro_r_teste.append(np.std(media_lag_r_teste))
        
    parametro.append(p)
    
dict = {'parametro': parametro,
        'mape_treino':  media_parametro_mape_treino, 
        'rmse_treino':  media_parametro_rmse_treino, 'std_rmse_treino':  std_parametro_rmse_treino,
        'r_quad_treino':media_parametro_r_treino,    
        'mape_teste':   media_parametro_mape_teste,  
        'rmse_teste':   media_parametro_rmse_teste,  'std_rmse_teste':   std_parametro_rmse_teste,
        'r_quad_teste': media_parametro_r_teste}
   
df_resultado_final = pd.DataFrame(dict)

dframes = [df_resultado_medio_g,df_resultado_final]
df_resultado_medio_g = pd.concat(dframes)

df_resultado_medio_g.round(2)

Agora é a vez do parâmetro: ph
Agora é a vez do parâmetro: coliformes
Agora é a vez do parâmetro: dbo
Agora é a vez do parâmetro: fosforo
Agora é a vez do parâmetro: od
Agora é a vez do parâmetro: solido
Agora é a vez do parâmetro: turbidez


,parametro,mape_treino,rmse_treino,std_rmse_treino,r_quad_treino,mape_teste,rmse_teste,std_rmse_teste,r_quad_teste
0,ph,0.39,0.00,0.00,0.94,2.32,0.02,0.00,0.21
1,coliformes,2.92,0.22,0.01,0.91,49.31,1.46,0.03,0.01
2,dbo,6.06,0.09,0.00,0.94,13.91,0.24,0.01,0.11
3,fosforo,24.22,0.04,0.00,0.93,74.70,0.11,0.00,0.02
4,od,29.08,0.06,0.00,0.93,36.45,0.12,0.00,0.13
5,solido,1.70,0.05,0.00,0.94,5.62,0.17,0.00,0.21
6,turbidez,4.25,0.07,0.00,0.93,12.85,0.24,0.00,0.01


In [5]:
df_resultado_medio_g.to_csv (r'/home/anderson/Downloads/src_experimentos_dissertação/resultados_log_csv/temporal_rf_var_selecionadas.csv', index = True, header=True)